# Modelo SRV

Este notebbok aplicará o algoritmo de SVR ao dataset obtido pelo xTB.

**Autor:** Edélio Gabriel Magalhães de Jesus.

<a id='sumario'></a>
## Sumário

- [1) Introdução](#intro)
  - [1.1) Support Vector Machine: Uma visão geral](#svm)
  - [1.2) Objetivo do notebook](#objetivo)
- [2) Desenvlvimento do modelo](#desenvolvimento)
  - [2.1) Bibliotecas necessárias](#bibliotecas)
  - [2.2) Leitura e primeiras visualizações do *dataset*](#leitura)
  - [2.3) Split dos dados](#split)

<a id='intro'></a>
## 1) Introdução 

<a id='svm'></a>
### **1.1) Support Vector Machine: Uma Visão Geral**

![gif](https://miro.medium.com/v2/resize:fit:1100/format:webp/0*2-EExC6mYeEMvtrw.gif)

**Support Vector Machine (SVM)** é um algoritmo de aprendizado supervisionado inicialmente desenvolvido para **classificação binária**. Seu objetivo é encontrar **o hiperplano ótimo** que separa os dados de duas classes em um espaço de \(n\) dimensões. Esse “ótimo” não significa apenas separar corretamente, mas sim **maximizar a margem**, isto é, a distância entre a fronteira de decisão e os pontos mais próximos a ela [1, 2, 3, 4]. Esse processo de maximização tem suporte da **Teoria de Aprendizado Estatístico (TAE)** [4, 5] - se quiser saber mais, clique [aqui](https://en.wikipedia.org/wiki/Statistical_learning_theory).

Esses pontos de cada classe que ficam mais próximos do hiperplano recebem o nome de **vetores de suporte** (*support vectors*) e são os elementos fundamentais que definem a solução final [1, 2, 3].  

Embora tenha surgido para problemas binários [4], o SVM pode ser estendido a **classificação multiclasse**. As duas estratégias mais comuns são [6]:  
- **One-vs-Rest (OvR)**: treina um classificador para cada classe contra todas as outras.  
- **One-vs-One (OvO)**: treina classificadores para cada par de classes e usa votação para decidir a predição final.  

Com o tempo, o conceito foi ampliado: hoje podemos considerar o **SVM** como sendo uma **família de modelos** baseados nessa mesma ideia de margem máxima, mas aplicados a diferentes contextos (classificação linear, não linear, regressão e até detecção de anomalias) [6].  

No **scikit-learn**, as principais variantes são:  
- **`SVC`** – Support Vector Classification, usado em classificação (linear ou com kernels).  
- **`LinearSVC`** – versão otimizada para problemas lineares de alta dimensionalidade.  
- **`NuSVC`** – alternativa ao `SVC` que usa o parâmetro `nu` em vez de `C` para controlar o número de vetores de suporte.  
- **`SVR`** – Support Vector Regression, aplica o mesmo princípio à regressão.  
- **`OneClassSVM`** – usado para detecção de anomalias/outliers. 


<a id='objetivo'></a>
### **1.2) Objetivo do notebook**

Neste notebook, implementaremos o algoritmo **`SVR`** em um modelo para resolver a tarefa de **regressão supervisionada**.

[Voltar ao topo](#sumario)

<a id='desenvolvimento'></a>
## 2) Desenvolvendo o modelo

<a id='bibliotecas'></a>
### **2.1) Bibliotecas e métodos necessários**

Antes de tudo, precisamos importar alguns módulos e funções específicas. São quatro bibliotecas principais:

1) **_pandas_**: para criação e manipulação de dataframes;  
2) **_numpy_**: para operações matemáticas e tratamento de arrays;  
3) **_plotly_**: para geração de gráficos interativos;  
4) **_sklearn_**: para implementação de modelos e ferramentas de *Machine Learning* (como divisão de dados, normalização e validação cruzada).  
5) **optuna**: para otimização dos hiperparâmetros

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, r2_score
import optuna as opt
import plotly.graph_objects as go
import plotly.express as px


<a id='leitura'></a>
### **2.2) Leitura e primeiras visualizações do *dataset***

In [14]:
SEMENTE_ALEATORIA = 367

df = pd.read_csv('./dataset_processing/xtb_dataset.csv').sample(frac=0.05, random_state=SEMENTE_ALEATORIA)

df.describe()

,Dipole,E_HOMO,E_LUMO,gap_HOMO-LUMO,ZPE,H,U,U0,G,Delta
count,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000,6458.000000
mean,3.215199,-10.470498,-5.762511,-4.707987,0.143425,-26.820574,-26.821518,-26.972640,-26.861253,-13.506291
std,1.799220,0.592685,2.317082,2.267831,0.032534,2.138999,2.138999,2.151887,2.140352,2.151887
min,0.000000,-15.970300,-11.115600,-14.350000,0.012600,-34.330713,-34.331658,-34.452850,-34.374238,-33.305032
25%,1.917250,-10.833825,-7.438400,-5.824675,0.120836,-28.230497,-28.231441,-28.376829,-28.270545,-14.597758
50%,2.990500,-10.501300,-6.344050,-4.084500,0.142842,-27.078704,-27.079648,-27.258813,-27.118890,-13.220118
75%,4.253750,-10.137950,-4.455900,-3.219950,0.165291,-25.758758,-25.759702,-25.881173,-25.797411,-12.102102
max,26.537000,-6.421500,2.805200,-0.181300,0.247998,-7.144001,-7.144946,-7.173899,-7.168830,-6.026081


A documentação disponível no *sklearn* [6] indica que, para conjunto de dados muito grandes, o algoritmo *LinearSVR* pode ser mais eficiente em termos de memória e tempo de execução. Contudo, é necessário ter certeza que a relação entre as variáveis é linear. Para isso, podemos conferir observando a `Matriz de Correlação`.

---

A **Matriz de Correlação** $\mathbf{C}$ é uma matriz simétrica contendo todas as possíveis correlações computadas para cada $x_i$, dois a dois (as correlações são os coeficientes de correlação de Pearson). Supondo que temos $(x_1, x_2, x_3)$, a matriz de correlação será:

$$
\mathbf{C} =
\left[\begin{array}{ccc}
\mathrm{Corr}(x_{1},x_{1})&\mathrm{Corr}(x_{1},x_{2})&\mathrm{Corr}(x_{1},x_{3})\\
\mathrm{Corr}(x_{2},x_{1})&\mathrm{Corr}(x_{2},x_{2})&\mathrm{Corr}(x_{2},x_{3})\\
\mathrm{Corr}(x_{3},x_{1})&\mathrm{Corr}(x_{3},x_{2})&\mathrm{Corr}(x_{3},x_{3})\\
\end{array} \right].
$$

---

In [3]:
corr = df.corr()

fig = px.imshow(corr, text_auto=True, aspect="auto",
                title="Matriz de Correlação")
fig.show()


Correlação próximas de **1** ou **-1** indicam uma forte relação linear entre a variável independente $X$ e o *target*.  
Com base nisso, observa-se que **apenas 4 dos 9 atributos** apresentam essa característica.  

Isso sugere que o conjunto de dados possui **relações predominantemente não lineares**, o que torna **inadequado o uso do algoritmo *LinearSVR*** — uma versão mais simples e menos custosa do *Support Vector Regressor*.  

Assim, **optamos pelo uso do *SVR* clássico**, que permite explorar diferentes *kernels* capazes de capturar **padrões mais complexos** nas relações entre as variáveis.

<a id='split'></a>
### **2.3) Split dos dados**

A etapa de *split* no tratamento de dados consiste em `dividir` o conjunto de dados disponível em subconjuntos distintos: *`treinamento`* e *`teste`*. Essa divisão é importante para que o modelo de aprendizado de máquina seja treinado em uma parte dos dados e, posteriormente, avaliado em dados que ele nunca viu, garantindo uma medição justa de seu desempenho e evitando o *overfitting* - o ajuste excessivo aos dados de treinamento - e o *underfitting* - subestimação dos dados.

In [4]:
TAMANHO_TESTE = 0.2
SEMENTE_ALEATORIA = 367

X = df.drop(columns=['Delta'])
y = df['Delta']

X_treino, X_teste, y_treino, y_teste =train_test_split(X, y, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA)

<a id='optuna'></a>
### **2.4) Otimização dos hiperparâmetros com o optuna**

#### `Modelo e espaço de busca`

A função `cria_instancia_modelo` abaixo serve para criar uma instância do modelo escolhido. Esta função recebe um objeto tipo *trial*; do `optuna`.

Observe que o dicionário `parametros` dentro desta função tem como chaves os nomes dos argumentos do modelo. Os valores dos argumentos, por sua vez, podem ser sorteados com as funções

-   `trial.suggest_int` (para números inteiros)
-   `trial.suggest_float` (para números reais) e
-   `trial.suggest_categorical` (para dados categóricos).

São com estas funções que delimitamos o **espaço de busca** dos hiperparâmetros do modelo.

Além dos parâmetros do modelo, estamos permitindo que o *optuna* decida se irá realizar os pré-processamentos de reduzir a dimensionalidade (via *PCA*).

In [11]:
def criar_instancia_modelo(trial):
    kernel_escolhido = trial.suggest_categorical("kernel", ["rbf", "poly", "linear", "sigmoid"])

    # gamma contínuo (apenas para kernels não lineares)
    if kernel_escolhido in ["rbf", "poly", "sigmoid"]:
        gamma = trial.suggest_float("gamma", 1e-4, 10, log=True)
    else:
        gamma = "scale"

    parametros_svr = {
        "C": trial.suggest_float("C", 1e-3, 1e3, log=True),
        "epsilon": trial.suggest_float("epsilon", 1e-4, 1.0, log=True),
        "kernel": kernel_escolhido,
        "degree": trial.suggest_int("degree", 2, 7) if kernel_escolhido == "poly" else 3,
        "gamma": gamma,
        "coef0": trial.suggest_float("coef0", 0.0, 10.0),
        "shrinking": trial.suggest_categorical("shrinking", [True, False]),
    }

    usar_pca = trial.suggest_categorical("usar_pca", [True, False])
    steps = [("normalizador", StandardScaler())]

    if usar_pca and kernel_escolhido in ["rbf", "poly", "sigmoid"]:
        n_comp = trial.suggest_float("variancia_pca", 0.85, 0.99)
        steps.append(("pca", PCA(n_components=n_comp, random_state=SEMENTE_ALEATORIA)))

    steps.append(("regressor", SVR(**parametros_svr)))
    modelo = Pipeline(steps=steps)
    return modelo


#### `Função objetivo`

A **função objetivo** em um problema de otimização é responsável por calcular a **métrica de desempenho** que será minimizada ou maximizada durante o processo.  

Neste caso, a métrica escolhida é o **RMSE (Root Mean Squared Error)**, obtido por meio de **validação cruzada**.

---

**Observação:**  
O *Python* (ou mais especificamente, algumas funções do `scikit-learn`) retorna valores **negativos** para métricas de erro quando configuradas para maximização.  
Por isso, é necessário **multiplicar por -1** antes de calcular a média dos valores de RMSE, garantindo que o algoritmo do Optuna minimize corretamente o erro.


In [12]:
from sklearn.model_selection import KFold, cross_val_score

def funcao_objetivo(trial, X, y, NUM_FOLDS=10):
    # Cria o modelo SVR usando o pipeline
    modelo = criar_instancia_modelo(trial)

    # KFold para regressão
    cv = KFold(
        n_splits=NUM_FOLDS,
        shuffle=True,
        random_state=SEMENTE_ALEATORIA
    )

    # Cross-validation usando RMSE como métrica
    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring="neg_root_mean_squared_error",  # para regressão
        cv=cv,
        n_jobs=-1
    )

    # Retorna a média do RMSE
    return -metricas.mean()


#### `Otimização`

A otimização dos hiperparâmetros é conduzida por meio da criação de um **objeto de estudo** com a função `create_study`.

No nosso caso, como a métrica de interesse é o **RMSE (Root Mean Squared Error)** — um indicador de erro —, quanto **menor** o seu valor, **melhor** o desempenho do modelo.  
Por isso, atribuímos o valor `"minimize"` ao argumento `direction`.

O parâmetro `study_name` define um **nome identificador** para o processo de busca, permitindo que possamos retomá-lo posteriormente.  
Já o argumento `storage` especifica o **local onde os resultados do estudo serão armazenados**, geralmente em um arquivo de banco de dados no formato SQLite (`.db`).

Por fim, o argumento `load_if_exists=True` instrui o Optuna a **verificar se já existe um estudo com o mesmo nome**.  
Caso exista, o novo processo será **incorporado ao estudo anterior**, permitindo continuar a otimização de forma cumulativa.


In [9]:
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

NOME_DO_ESTUDO = "svr_xtb_dataset_2"
NUM_FOLDS = 10
NUM_TENTATIVAS = 300

# Criar ou carregar estudo existente
objeto_de_estudo = create_study(
    direction="minimize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
    sampler=TPESampler(seed=SEMENTE_ALEATORIA),
    pruner=HyperbandPruner(min_resource=1, max_resource=10, reduction_factor=3) 
)

# Forçar algumas combinações iniciais
objeto_de_estudo.enqueue_trial({"usar_pca": False})
objeto_de_estudo.enqueue_trial({"usar_pca": True})

# Função-objetivo parcial
def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-10-25 21:10:43,250] Using an existing study with name 'svr_xtb_dataset' instead of creating a new one.
[I 2025-10-25 21:11:20,274] Trial 54 finished with value: 0.006211079712604486 and parameters: {'kernel': 'poly', 'C': 30.767013800489934, 'epsilon': 0.01038687192142026, 'degree': 3, 'gamma': 'auto', 'coef0': 0.1796200571206607, 'shrinking': True, 'usar_pca': False}. Best is trial 20 with value: 0.0032931874855121244.
[I 2025-10-25 23:11:48,253] Trial 55 finished with value: 3.7940842870676854 and parameters: {'kernel': 'poly', 'C': 33.74320410366235, 'epsilon': 0.01775495785261328, 'degree': 4, 'gamma': 'auto', 'coef0': 0.003450700628152297, 'shrinking': True, 'usar_pca': True, 'variancia_pca': 0.929004303375524}. Best is trial 20 with value: 0.0032931874855121244.
[I 2025-10-25 23:11:51,156] Trial 56 finished with value: 0.006223972376598386 and parameters: {'kernel': 'poly', 'C': 14.023481621062867, 'epsilon': 0.016285704562603635, 'degree': 2, 'gamma': 'auto', 'coef0': 0.

In [10]:
melhor_trial = objeto_de_estudo.best_trial
print(f"Score médio: {melhor_trial.value:.4f}")
print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Score médio: 0.0028
Número do melhor trial: 76
Parâmetros do melhor trial: {'kernel': 'linear', 'C': 98.4060322888323, 'epsilon': 0.010039007818614078, 'gamma': 'scale', 'coef0': 0.006460562236304514, 'shrinking': True, 'usar_pca': False}


<a id='ref'></a>
## Referências

[1] **Livro**: RASCHKA, Sebastian; MIRJALILI, Vahid. *Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow 2*. 3. ed. Packt, [s.d.].  

[2] **Vídeo** do canal StatQuest with Josh Starmer - [Support Vector Machines Part 1 (of 3): Main Ideas!!!](https://youtu.be/efR1C6CvhmE?si=yfCtwqnRPt1FEO1O)

[3] **Vídeo** do canal Hashtag Programação - [SVM (Support Vector Machine) - Algoritmos de Aprendizado de Máquinas](https://youtu.be/Y-WFZd9_gtE?si=5CMIM6krA9kQBNNy)

[4] **Livro**: FACELI, Katti; LORENA, et al.. Inteligência artificial: uma abordagem de aprendizado de máquina. 2. ed. Rio de Janeiro: LTC, 2021. 400 p.

[5] **Site** Wikipedia - [Teoria de Aprendizado Estatístico](https://en.wikipedia.org/wiki/Statistical_learning_theory)

[6] **Documentação** do sklearn - [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html#svm)